In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.stats import norm
from scipy.stats import skew, kurtosis, skewtest
from scipy.stats.mstats import gmean
from scipy.stats import jarque_bera

In [2]:
# Wikipedia URL for IPSA components
url = 'https://es.wikipedia.org/wiki/%C3%8Dndice_de_Precio_Selectivo_de_Acciones'

# Extract tables from the Wikipedia page
tables = pd.read_html(url)

# Display the first table (if it exists)
print(tables[0])

    N.º   Nemotécnico                             Empresa
0     1       AGUAS-A                       Aguas Andinas
1     2      ANDINA-B                Embotelladora Andina
2     3           BCI      Banco de Crédito e Inversiones
3     4    BSANTANDER               Banco Santander Chile
4     5           CAP                                 CAP
5     6           CCU      Compañía de Cervecerías Unidas
6     7    CENCOSHOPP              Cencosud Shopping S.A.
7     8      CENCOSUD                       Cencosud S.A.
8     9         CHILE                      Banco de Chile
9    10          CMPC                       Empresas CMPC
10   11        COLBUN                              Colbún
11   12    CONCHATORO                  Viña Concha y Toro
12   13         COPEC                               Copec
13   14           ECL                               Engie
14   15        ENELAM                       Enel Américas
15   16     ENELCHILE                          Enel Chile
16   17       

In [3]:
df_idx=[i + '.SN' for i in tables[0]['Nemotécnico']]
tables[0]['Nemotécnico']=[i + '.SN' for i in tables[0]['Nemotécnico']]

In [4]:
#Se validará el funcionamiento de los tickers proporcionados, identificando y filtrando aquellos que no retornan información válida.

def validate_tickers(ticker_list):
    """
    Validate a list of tickers to check which ones are valid in yfinance.

    Parameters:
        ticker_list (list): List of tickers to validate.

    Returns:
        valid_tickers (list): A list of valid tickers.
        invalid_tickers (list): A list of invalid tickers.
    """
    valid_tickers = []
    invalid_tickers = []

    for ticker in ticker_list:
        try:
            # Attempt to fetch basic info for the ticker
            stock = yf.Ticker(ticker)

            # Try fetching a common piece of information like "info" or historical data
            if stock.info.get("bookValue") is None:
                print(f"Ticker {ticker} is invalid (no data in info).")
                invalid_tickers.append(ticker)
            else:
                #print(f"Ticker {ticker} is valid.")
                valid_tickers.append(ticker)
        except Exception as e:
            # Handle specific errors like 404 Client Errors
            print(f"An error occurred with ticker {ticker}: {e}")
            invalid_tickers.append(ticker)

    return valid_tickers, invalid_tickers


In [5]:
#ticker list
ticker_list = df_idx

# Validate tickers
valid_tickers, invalid_tickers = validate_tickers(ticker_list)

print("\nValid Tickers:", valid_tickers)
print("Invalid Tickers:", invalid_tickers)

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CENCOSHOPP.SN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CENCOSHOPP.SN&crumb=XBUWaGswFeX


Ticker CENCOSHOPP.SN is invalid (no data in info).
Ticker ORO%20BLANCO.SN is invalid (no data in info).

Valid Tickers: ['AGUAS-A.SN', 'ANDINA-B.SN', 'BCI.SN', 'BSANTANDER.SN', 'CAP.SN', 'CCU.SN', 'CENCOSUD.SN', 'CHILE.SN', 'CMPC.SN', 'COLBUN.SN', 'CONCHATORO.SN', 'COPEC.SN', 'ECL.SN', 'ENELAM.SN', 'ENELCHILE.SN', 'ENTEL.SN', 'FALABELLA.SN', 'IAM.SN', 'ITAUCL.SN', 'LTM.SN', 'MALLPLAZA.SN', 'PARAUCO.SN', 'QUINENCO.SN', 'RIPLEY.SN', 'SMU.SN', 'SONDA.SN', 'SQM-B.SN', 'VAPORES.SN']
Invalid Tickers: ['CENCOSHOPP.SN', 'ORO%20BLANCO.SN']


In [6]:
# Elegir Acciones por agregar al Protafolio y Seleccionar periodo de muestra
start_date = '2023-01-01'
end_date = '2024-01-01'

data = yf.download(valid_tickers, start=start_date, end=end_date)['Adj Close']

[*********************100%***********************]  28 of 28 completed


In [8]:
data.head()

Ticker,AGUAS-A.SN,ANDINA-B.SN,BCI.SN,BSANTANDER.SN,CAP.SN,CCU.SN,CENCOSUD.SN,CHILE.SN,CMPC.SN,COLBUN.SN,...,ITAUCL.SN,LTM.SN,MALLPLAZA.SN,PARAUCO.SN,QUINENCO.SN,RIPLEY.SN,SMU.SN,SONDA.SN,SQM-B.SN,VAPORES.SN
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,172.607056,1808.050415,17540.474609,29.868242,6572.856934,5179.000000,1247.427490,76.659500,1393.320679,87.749458,...,8018.590332,5.830,976.522278,990.968567,2125.364014,121.741264,100.717300,302.232056,65694.664062,67.298927
2023-01-04,172.132065,1790.581421,17612.955078,29.555252,6535.164062,5111.912109,1234.216797,75.359428,1364.824463,86.979637,...,8018.590332,5.720,967.897461,960.240845,2110.396484,120.999161,100.540947,298.602081,66742.562500,66.969078
2023-01-05,172.840057,1781.846558,17815.902344,29.760931,6361.816406,5198.843262,1230.187866,75.045616,1413.818237,86.229820,...,7932.830566,5.481,946.153320,969.843262,2125.364014,120.716446,102.621796,297.920868,67493.500000,67.898643
2023-01-06,171.612274,1777.479248,17960.865234,29.921898,6409.649902,5326.026367,1214.728638,74.857330,1469.810913,87.659477,...,7992.862305,5.521,943.240051,965.138062,2125.364014,122.695404,101.193405,336.404388,67495.500000,67.878654
2023-01-09,170.384476,1738.173950,17685.435547,30.118637,6615.332520,5327.349609,1218.007812,74.866295,1459.812134,88.979179,...,7962.846191,5.279,964.830811,960.240845,2087.571289,123.243149,102.277916,347.117065,68853.359375,67.948624


In [16]:
# Agrupar por ticker y sumar el volumen
bottom_5=data.mean().sort_values().reset_index().head(5)
top_5=data.mean().sort_values().reset_index().tail(5)

#revisaremos el VaR de las 5 mas transadas y las 5 menos transadas
ticker_list=bottom_5['Ticker'].tolist()+top_5['Ticker'].tolist()

In [30]:
data_close = yf.download(ticker_list,start=start_date,end=end_date)['Adj Close']

[*********************100%***********************]  10 of 10 completed


In [31]:
returns = data_close.pct_change()
returns = returns.dropna()

In [60]:
df_1=pd.DataFrame(columns=['Ticker', 'Semi_desviacion', 'Des_Estandar'])

In [61]:
for i,ticker in enumerate(returns.columns):
    # Calcular la media de los retornos
  mean_return = returns[ticker].mean()
  std_return =  returns[ticker].std()
    # Filtrar solo los retornos negativos (desviaciones negativas)
  negative_deviations = returns[ticker][returns[ticker] < mean_return]
    # Calcular la semidesviación
  semi_deviation = np.sqrt(np.sum((negative_deviations - mean_return) ** 2) / len(returns))
  df_1.loc[i,['Ticker', 'Semi_desviacion', 'Des_Estandar']]=ticker, semi_deviation, std_return

El Valor en Riesgo (VaR) es una técnica financiera que se utiliza para medir el riesgo de una inversión, cartera o empresa en un periodo de tiempo determinado\
Se define como 'la máxima pérdida esperada en un periodo de tiempo y con un nivel de confianza dados, encondiciones normales de mercado'

Método de simulación Histórica: Es un método aplicable tanto a carteras
lineales como no lineales, debido a que es un método no paramétrico, que
no depende de ninguna de las hipótesis sobre distribuciones de probabilidad
subyacente y, por lo tanto, permite capturar el fenómeno de las colas
gruesas al mismo tiempo que elimina la necesidad de estimar y trabajar con
volatilidades y correlaciones, evitando en gran medida el riesgo de
modelización.\
Ventajas: Las ventajas fueron mencionadas en su definición, y hace que
este método sea preferible al de matriz varianza-covarianza, especialmente
cuando se trata del cálculo del VaR para carteras de instrumentos no
lineales.\
Desventajas: La principal desventaja del método de simulación histórica
viene dada por las características de los datos utilizados, que supone que
ningún evento que no haya ocurrido en el pasado podrá ocurrir en el futuro.

In [62]:
niveles_confianza = 0.95  # Niveles de confianza: 95% y 99%
for i,ticker in enumerate(returns.columns):
    var_historico = np.percentile(returns[ticker], (1 - niveles_confianza) * 100)
    df_1.loc[i,['VaRH_95']]=var_historico

,Ticker,Semi_desviacion,Des_Estandar,VaRH_95
0,BCI.SN,0.010161,0.015264,-0.022869
1,BSANTANDER.SN,0.009714,0.013332,-0.023236
2,CAP.SN,0.013555,0.019538,-0.032241
3,CHILE.SN,0.00832,0.012425,-0.018301
4,COPEC.SN,0.010615,0.01577,-0.023590
5,ENELCHILE.SN,0.013186,0.018463,-0.027329
6,ITAUCL.SN,0.007413,0.01244,-0.014956
7,LTM.SN,0.015026,0.022768,-0.034064
8,SQM-B.SN,0.021029,0.02901,-0.040364
9,VAPORES.SN,0.020877,0.027081,-0.026692


In [68]:
# VaR método de varianza-covarianza
for i,ticker in enumerate(returns.columns):
  mean_return = returns[ticker].mean()
  std_dev = returns[ticker].std()

  # Calcular el VaR con un nivel de confianza del 95 % utilizando la puntuación Z
  confidence_level = 0.95
  z_score = norm.ppf( 1 - confidence_level)
  VaR_varianza_covarianza = mean_return + z_score*std_dev
  df_1.loc[i,['Var_Cov']]=VaR_varianza_covarianza

df_1

,Ticker,Semi_desviacion,Des_Estandar,VaRH_95,Var_Cov
0,BCI.SN,0.010161,0.015264,-0.022869,-0.023888
1,BSANTANDER.SN,0.009714,0.013332,-0.023236,-0.020527
2,CAP.SN,0.013555,0.019538,-0.032241,-0.031577
3,CHILE.SN,0.00832,0.012425,-0.018301,-0.019141
4,COPEC.SN,0.010615,0.01577,-0.023590,-0.025657
5,ENELCHILE.SN,0.013186,0.018463,-0.027329,-0.028439
6,ITAUCL.SN,0.007413,0.01244,-0.014956,-0.020307
7,LTM.SN,0.015026,0.022768,-0.034064,-0.035186
8,SQM-B.SN,0.021029,0.02901,-0.040364,-0.048193
9,VAPORES.SN,0.020877,0.027081,-0.026692,-0.045038


In [70]:

for i,ticker in enumerate(returns.columns):
  # Simular retornos futuros usando Monte Carlo
  num_simulations = 10000
  simulation_horizon = 252   # Número de días de negociación en un año
  simulated_returns = np.random.normal(np.mean(returns[ticker]), np.std(returns[ticker]), (simulation_horizon, num_simulations))

  # Calcular los valores simulados de la cartera
  initial_investment = 1000000   # $1,000,000
  portfolio_values = initial_investment * np.exp(np.cumsum(simulated_returns, axis= 0 ))

  # Calcular los retornos de la cartera
  portfolio_returns = portfolio_values[- 1 ] / portfolio_values[ 0 ] - 1

  # Calcular el VaR al 95% de nivel de confianza
  confidence_level = 0.95
  VaR_monte_carlo = np.percentile(portfolio_returns, ( 1 - confidence_level) * 100 )
  df_1.loc[i,['Monte_Carlo']]=VaR_monte_carlo



In [75]:
df_2 = df_1.drop('Ticker', axis=1)

El valor en riesgo (VaR) es una métrica vital para evaluar el riesgo financiero, ya que ofrece una cuantificación clara de las pérdidas potenciales. Al implementar el VaR con Python, los inversores y los gestores de riesgos pueden obtener información útil sobre el perfil de riesgo de su cartera. Los ejemplos proporcionados con el método histórico, el método de varianza-covarianza y la simulación de Monte Carlo demuestran lo versátil y potente que puede ser el VaR en la gestión del riesgo financiero.

In [84]:
for i,ticker in enumerate(returns.columns):
  # Supongamos que tenemos los retornos diarios de un activo
  #np.random.seed(42)

  # Calcular parámetros básicos
  mean_return = np.mean(returns[ticker])
  std_return = np.std(returns[ticker])
  skewness = skew(returns[ticker])
  kurt = kurtosis(returns[ticker], fisher=True) + 3  # Curtosis total (no solo excesiva)

  # Nivel de confianza
  confidence_level = 0.05
  z_alpha = -1.645  # Para el 95% de confianza

  # Ajuste Cornish-Fisher
  z_adjusted = (
      z_alpha
      + (z_alpha**2 - 1) * skewness / 6
      + (z_alpha**3 - 3 * z_alpha) * (kurt - 3) / 24
      - (2 * z_alpha**3 - 5 * z_alpha) * (skewness**2) / 36
  )

  # Calcular VaR usando Cornish-Fisher
  var_cornish_fisher = -z_adjusted * std_return + mean_return
  df_1.loc[i,['Cornish_Fisher']]=var_cornish_fisher


In [86]:
df_1

,Ticker,Semi_desviacion,Des_Estandar,VaRH_95,Var_Cov,Monte_Carlo,Cornish_Fisher
0,BCI.SN,0.010161,0.015264,-0.022869,-0.023888,-0.083227,0.023112
1,BSANTANDER.SN,0.009714,0.013332,-0.023236,-0.020527,-0.001179,0.024356
2,CAP.SN,0.013555,0.019538,-0.032241,-0.031577,-0.306934,0.031888
3,CHILE.SN,0.00832,0.012425,-0.018301,-0.019141,0.001451,0.020009
4,COPEC.SN,0.010615,0.01577,-0.023590,-0.025657,-0.290598,0.023596
5,ENELCHILE.SN,0.013186,0.018463,-0.027329,-0.028439,0.007549,0.033064
6,ITAUCL.SN,0.007413,0.01244,-0.014956,-0.020307,-0.249265,0.005235
7,LTM.SN,0.015026,0.022768,-0.034064,-0.035186,-0.013512,0.034546
8,SQM-B.SN,0.021029,0.02901,-0.040364,-0.048193,-0.584480,0.049294
9,VAPORES.SN,0.020877,0.027081,-0.026692,-0.045038,-0.567288,0.038011


  Volumen Bajo (Primeros 5 Tickers):\
        Los tickers con menor volumen de transacciones (BCI.SN, BSANTANDER.SN, CAP.SN, CHILE.SN, COPEC.SN) presentan:\
            Valores más bajos de semi-desviación y desviación estándar en comparación con los tickers de mayor volumen.\
            Por ejemplo, CHILE.SN tiene una semi-desviación de 0.0083 y desviación estándar de 0.0124, mucho menor que SQM-B.SN (0.0210 y 0.0290, respectivamente).\
        Esto sugiere que los tickers con menor volumen tienden a ser menos volátiles, lo que puede reflejar una menor liquidez y actividad de mercado que suaviza las fluctuaciones de precios.

  Volumen Alto (Últimos 5 Tickers):\
        Los tickers con mayor volumen (ENELCHILE.SN, ITAUCL.SN, LTM.SN, SQM-B.SN, VAPORES.SN) tienen:\
            Valores de semi-desviación y desviación estándar notablemente más altos.\
            Por ejemplo, VAPORES.SN tiene una semi-desviación de 0.0208 y desviación estándar de 0.0271, casi el doble que los tickers de menor volumen.\
        Esto indica que los activos con mayor volumen son más propensos a fluctuaciones significativas, probablemente debido a una mayor participación de inversores y reacciones más rápidas a eventos de mercado.

Análisis del VaR con Volumen

  VaR Histórico (VaRH_95):\
        Los tickers con mayor volumen (SQM-B.SN, VAPORES.SN) presentan los valores de VaR histórico más conservadores (−0.0404 y −0.0266), reflejando una mayor probabilidad de pérdidas extremas.
        En contraste, los tickers de menor volumen tienen VaR más moderados (−0.0183 para CHILE.SN), lo que indica un menor riesgo percibido.

  Cornish-Fisher:\
        Los ajustes de asimetría y curtosis son más marcados para los tickers de mayor volumen, lo que se traduce en valores más altos (0.0492 para SQM-B.SN, 0.0380 para VAPORES.SN). Esto refleja una mayor exposición a eventos de cola en activos con alta actividad.

Nivel de Confianza (95%)

  Todas las métricas de VaR se calcularon con un nivel de confianza del 95%, lo que significa que existe un 5% de probabilidad de que las pérdidas superen los valores estimados.\
  Esto subraya que los valores de VaR no representan pérdidas máximas absolutas, sino un umbral basado en datos históricos y/o supuestos estadísticos.\
  La elección del 95% de confianza puede ser adecuada para análisis de riesgo moderado, pero un nivel del 99% podría ser más útil para escenarios conservadores o activos de mayor volumen como SQM-B.SN y VAPORES.SN.\

Conclusión Adicional sobre el Volumen y el Riesgo

  Tickers de Volumen Bajo:\
        Presentan menor riesgo a la baja y menos volatilidad general, lo que los hace más atractivos para inversores con baja tolerancia al riesgo.

  Tickers de Volumen Alto:\
        Aunque presentan mayores riesgos y volatilidad, estos activos también podrían ofrecer mayores oportunidades de retorno, ya que su mayor volumen suele ser reflejo de alta liquidez y participación activa del mercado.

  Diversificación:\
        Dada la diferencia en los perfiles de riesgo entre los tickers de alto y bajo volumen, una estrategia diversificada podría aprovechar las características de ambos grupos, combinando estabilidad con potencial de retorno.